In [ ]:
import pandas as pd
import ast
import requests

df = pd.read_csv('KDX+국가교통데이터오픈마켓+해양수산_20250730_6.csv')
df['tags_list'] = df['tags'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

log_file = open('create_category.log', 'w')

# 계층 분류 구조
category_tree = {
  "소비/유통": {
    "상품": ["유통", "판매", "쇼핑", "구매", "온라인몰", "물류"],
    "소비행태": ["소비", "소비트렌드", "소비자", "마케팅"],
    "거래정보": ["구매후기", "리뷰", "리뷰정보"]
  },
  "공간/위치": {
    "인구이동": ["유동인구", "방문인구", "직장인구", "서비스인구", "주거인구"],
    "위치기반": ["위치데이터", "위치정보", "입출항", "스케줄"],
    "관광": ["관광", "관광지", "여행"]
  },
  "데이터분석": {
    "빅데이터": ["빅데이터", "데이터", "분석", "AI", "텍스트마이닝"],
    "미디어분석": ["SNS", "소셜", "블로그", "트위터", "인스타그램", "미디어"],
    "자연어처리": ["감성분석", "의견분석", "텍스트"]
  },
  "산업/기술": {
    "IT기술": ["IoT", "클라우드", "AI", "인공지능", "플랫폼"],
    "해양/수산": ["어업", "양식", "해양", "어선정보", "조업", "실습선", "어선", "여객선", "선박"],
    "에너지": ["풍력", "태양광", "신재생에너지"]
  },
  "공공/행정": {
    "정책": ["정부", "행정", "공공데이터"],
    "통계": ["통계청", "자료", "데이터개방"],
    "지역": ["광역시", "시군구", "지역"]
  }
}

# 소분류 -> (대분류, 중분류) 맵핑
subcategory_map = {}
for big, mids in category_tree.items():
    for mid, subs in mids.items():
        for sub in subs:
            subcategory_map[sub] = (big, mid, sub)

# 결과 열 초기화
df['대분류'] = None
df['중분류'] = None
df['소분류'] = None

missings = {}

count = 0
for idx, row in df.iterrows():
    tags = row['tags_list']
    found = False      
    data_name = df.at[idx, 'name']
    data_description = df.at[idx, 'description']
    req_json = {
                "model": "exaone3.5:32b",  # or any model you have downloaded
                "prompt": f"""다음과 같이 데이터 마켓플레이스에서 제공하는 데이터의 태그 정보가 주어져있다
{str(tags)}
데이터의 이름은 다음과 같다
{str(data_name)}
데이터의 설명은 다음과 같다
{str(data_description)}

이들을 공통적으로 지칭할 수 있는 분류 정보를 다음 중에서 딱 하나만 고르시오
'유통', '판매', '쇼핑', '구매', '온라인몰', '물류', '소비', '소비트렌드', '소비자', '마케팅', '구매후기', '리뷰', '리뷰정보', '유동인구', '방문인구', '직장인구', '서비스인구', '주거인구', '위치데이터', '위치정보', '입출항', '스케줄', '관광', '관광지', '여행', '빅데이터', '데이터', '분석', 'AI', '텍스트마이닝', 'SNS', '소셜', '블로그', '트위터', '인스타그램', '미디어', '감성분석', '의견분석', '텍스트', 'IoT', '클라우드', '인공지능', '플랫폼', '어업', '양식', '해양', '어선정보', '조업', '실습선', '어선', '여객선', '선박', '풍력', '태양광', '신재생에너지', '정부', '행정', '공공데이터', '통계청', '자료', '데이터개방', '광역시', '시군구', '지역'
반드시 이들 중에서 하나를 골라야 하며 부연설명은 하지말고 분류 정보만 말하시오
적절한 분류가 없다면 '기타' 로 응답하시오""",
                    "stream": False     # Set to True for streaming responses
                }        
    while True:
        try:
            response = requests.post("http://localhost:11434/api/generate",json=req_json)
            data = response.json()
            sub_category = data['response']
            
            print(data['response'], tags)
            
            if sub_category == '기타':
                big, mid, sub = '기타', '기타', '기타'
            else:        
                big, mid, sub = subcategory_map[sub_category]                
                
            log_file.write(f"{data['response']} {tags} / {big} / {mid} / {sub}\n")
            log_file.flush()
            df.at[idx, '대분류'] = big
            df.at[idx, '중분류'] = mid
            df.at[idx, '소분류'] = sub
            break
        except Exception as e:
            print('Error', idx, sub_category)
            log_file.write(f"Error {idx} {sub_category}\n")
            log_file.flush()
            req_json = {
                "model": "exaone3.5:32b",  # or any model you have downloaded
                "prompt": f"""당신이 방금 분류한 '{sub_category}' 는 정의된 분류상에 없으므로 다시 진행한다
다음과 같이 데이터 마켓플레이스에서 제공하는 데이터의 태그 정보가 주어져있다
{str(tags)}
데이터의 이름은 다음과 같다
{str(data_name)}
데이터의 설명은 다음과 같다
{str(data_description)}

이들을 공통적으로 지칭할 수 있는 분류 정보를 다음 중에서 딱 하나만 고르시오
'유통', '판매', '쇼핑', '구매', '온라인몰', '물류', '소비', '소비트렌드', '소비자', '마케팅', '구매후기', '리뷰', '리뷰정보', '유동인구', '방문인구', '직장인구', '서비스인구', '주거인구', '위치데이터', '위치정보', '입출항', '스케줄', '관광', '관광지', '여행', '빅데이터', '데이터', '분석', 'AI', '텍스트마이닝', 'SNS', '소셜', '블로그', '트위터', '인스타그램', '미디어', '감성분석', '의견분석', '텍스트', 'IoT', '클라우드', '인공지능', '플랫폼', '어업', '양식', '해양', '어선정보', '조업', '실습선', '어선', '여객선', '선박', '풍력', '태양광', '신재생에너지', '정부', '행정', '공공데이터', '통계청', '자료', '데이터개방', '광역시', '시군구', '지역'
당신이 방금 분류한 '{sub_category}' 는 위에 정의된 분류상에 없다
반드시 위에 언급한 내용 중에서 하나를 골라야 하며 부연설명은 하지말고 분류 정보만 말하시오
적절한 분류가 없다면 '기타' 로 응답하시오""",
                    "stream": False     # Set to True for streaming responses
                }

df.to_csv('KDX+국가교통데이터오픈마켓+해양수산_20250731.csv', index=False)